[PythonでRESAS APIを使ってデータをダウンロード](https://note.nkmk.me/python-resas-api-download/)

[Qiita: RESAS-APIをつかってみました](https://qiita.com/kk300/items/993a869a0119f3935bb8)


In [1]:
import json
import urllib.request
import pprint
import pandas as pd

In [14]:
with open('./resas_api.json') as f:
    api_key = json.load(f)

print(api_key)
print(type(api_key))

{'X-API-KEY': 'XBrssyMmKjnKtDxLLQLdcpEJZYKrpgbN6RqPAaHr'}
<class 'dict'>


In [15]:
url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'
req = urllib.request.Request(url, headers=api_key)

In [16]:
with urllib.request.urlopen(req) as response:
    data = response.read()

print(type(data))

<class 'bytes'>


In [17]:
d = json.loads(data.decode())
pprint.pprint(d)

{'message': None,
 'result': [{'prefCode': 1, 'prefName': '北海道'},
            {'prefCode': 2, 'prefName': '青森県'},
            {'prefCode': 3, 'prefName': '岩手県'},
            {'prefCode': 4, 'prefName': '宮城県'},
            {'prefCode': 5, 'prefName': '秋田県'},
            {'prefCode': 6, 'prefName': '山形県'},
            {'prefCode': 7, 'prefName': '福島県'},
            {'prefCode': 8, 'prefName': '茨城県'},
            {'prefCode': 9, 'prefName': '栃木県'},
            {'prefCode': 10, 'prefName': '群馬県'},
            {'prefCode': 11, 'prefName': '埼玉県'},
            {'prefCode': 12, 'prefName': '千葉県'},
            {'prefCode': 13, 'prefName': '東京都'},
            {'prefCode': 14, 'prefName': '神奈川県'},
            {'prefCode': 15, 'prefName': '新潟県'},
            {'prefCode': 16, 'prefName': '富山県'},
            {'prefCode': 17, 'prefName': '石川県'},
            {'prefCode': 18, 'prefName': '福井県'},
            {'prefCode': 19, 'prefName': '山梨県'},
            {'prefCode': 20, 'prefName': '長野県'},
          

Store the data as pandas data frame

In [18]:
import pandas as pd
df = pd.io.json.json_normalize(d['result'])
print(df.head())

   prefCode prefName
0         1      北海道
1         2      青森県
2         3      岩手県
3         4      宮城県
4         5      秋田県


In [19]:
df.head(10)

,prefCode,prefName
0,1,北海道
1,2,青森県
2,3,岩手県
3,4,宮城県
4,5,秋田県
5,6,山形県
6,7,福島県
7,8,茨城県
8,9,栃木県
9,10,群馬県


APIの一覧は[ここから](https://opendata.resas-portal.go.jp/docs/api/v1/cities.html)見られる。例えば市区町村データの場合、

```
GET  api/v1/cities
```
と記載があるので、URLの最後の文字列をこれに変更する。サンプルのところにあるように、

```html
GET https://opendata.resas-portal.go.jp/api/v1/cities?prefCode=1
```

とあるので、都道府県コードをパラメータとして指定する。

In [24]:
url = 'https://opendata.resas-portal.go.jp/api/v1/cities?prefCode=13'
req = urllib.request.Request(url, headers=api_key)
with urllib.request.urlopen(req) as response:
    data = response.read()

    
d = json.loads(data.decode())
cities = pd.io.json.json_normalize(d['result'])
cities.head(5)

,bigCityFlag,cityCode,cityName,prefCode
0,3,13101,千代田区,13
1,3,13102,中央区,13
2,3,13103,港区,13
3,3,13104,新宿区,13
4,3,13105,文京区,13


### 昼間・夜間人口

In [26]:
url = 'https://opendata.resas-portal.go.jp/api/v1/townPlanning/commuteSchool/areaPopulationCircle?prefecture_cd=13&city_cd=13101&mode=2&year=2015'
req = urllib.request.Request(url, headers=api_key)
with urllib.request.urlopen(req) as response:
    data = response.read()

    
d = json.loads(data.decode())
pop = pd.io.json.json_normalize(d['result'])
pop.head(5)

,city,dayNightRate,mode,nightData,nightDataSum,noonData,noonDataSum,pref,year
0,13101,1460.58,2,"[{'value': 47647, 'code': '13101', 'name': '東京...",58406,"[{'value': 61909, 'code': '14100', 'name': '神奈...",853068,13,2015


In [29]:
pop.nightData

0    [{'value': 47647, 'code': '13101', 'name': '東京...
Name: nightData, dtype: object